In [1]:
import pandas as pd

# **BONUS TRACK 1**

Una vez creada la base de datos, realiza consultas a esa base de datos:

In [2]:
# Nos conectamos a la base de datos que hemos creado.
import psycopg2
# Importamos os y dotenv para poder recuperar variables del archivo .env
import os
from dotenv import load_dotenv
# Cargamos variables de entorno
load_dotenv()
dbname = os.getenv("dbname")
user =  os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")
# Configurar la conexión
conn = psycopg2.connect(
    dbname = dbname,
    user = user,
    password = password,
    host = host,
    port = port)
# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()

1. Cuantos hoteles tiene la base de datos

In [3]:
query1 = """ SELECT count(id_hotel) 
            FROM hoteles;"""
cur.execute(query1)
resultado1 = cur.fetchall()
resultado1

[(29,)]

In [4]:
print(f"La base de datos tiene {resultado1[0][0]} hoteles")

La base de datos tiene 29 hoteles


2. Cuantas reservas se han hecho

In [5]:
query2 = """ SELECT count(id_reserva) 
            FROM reservas;"""
cur.execute(query2)
resultado2 = cur.fetchall()
resultado2

[(15000,)]

In [6]:
print(f"Se han realizado {resultado2[0][0]} reservas en total")

Se han realizado 15000 reservas en total


3. Identifica los 10 clientes que más se han gastado

In [7]:
query3 = """ SELECT c.id_cliente, nombre, apellido, sum(r.precio_noche) AS total
            FROM clientes AS c
            JOIN reservas AS r on c.id_cliente = r.id_cliente
            GROUP BY c.id_cliente
            ORDER BY sum(r.precio_noche) DESC
            LIMIT 10
              ;"""
cur.execute(query3)
# Organizamos el resultado en un dataframe para verlo mejor
resultado3 = pd.DataFrame(cur.fetchall(), columns=["id_cliente", "nombre", "apellido", "gasto_total"])
resultado3

,id_cliente,nombre,apellido,gasto_total
0,cliente_205,Ceferino,Sosa,888.93
1,cliente_255,Leandra,Castañeda,823.44
2,cliente_2576,Modesta,Heras,805.08
3,cliente_1567,Clarisa,Coll,795.63
4,cliente_6807,Abigaíl,Ayala,791.89
5,cliente_6190,Domingo,Zabaleta,778.89
6,cliente_3975,Consuela,Folch,749.41
7,cliente_3780,Samuel,Arco,696.95
8,cliente_11129,Ángeles,Nuñez,696.65
9,cliente_2798,Ariadna,Miguel,695.66


4. Identifica el hotel de la competencia y el hotel de nuestra marca que más han recaudado para esas fechas

In [8]:
conn.rollback()

In [9]:
query4_c = """ SELECT h.nombre_hotel, sum(r.precio_noche)
                FROM hoteles AS h
                JOIN reservas AS r on h.id_hotel = r.id_hotel
                GROUP BY h.nombre_hotel, h.competencia
                HAVING h.competencia = True
                ORDER BY sum(r.precio_noche) DESC
                LIMIT 1
              ;"""
cur.execute(query4_c)
resultado4_c = cur.fetchall()

query4_m = """ SELECT h.nombre_hotel, sum(r.precio_noche)
                FROM hoteles AS h
                JOIN reservas AS r on h.id_hotel = r.id_hotel
                GROUP BY h.nombre_hotel, h.competencia
                HAVING h.competencia = False
                ORDER BY sum(r.precio_noche) DESC
                LIMIT 1
              ;"""
cur.execute(query4_m)
resultado4_m = cur.fetchall()


In [10]:
print(f"El hotel de la competencia que más ha recaudado es el {resultado4_c[0][0]} con {resultado4_c[0][1]} €") 
print(f"El hotel de nuestra marca que más ha recaudado es el {resultado4_m[0][0]} con {resultado4_m[0][1]} €")

El hotel de la competencia que más ha recaudado es el ibis Madrid Centro las Ventas con 94844.0 €
El hotel de nuestra marca que más ha recaudado es el Hotel Monte Verde con 152136.80952380956 €


5. Identifica cuantos eventos hay

In [11]:
query5 = """ SELECT count(id_evento) 
            FROM eventos;"""
cur.execute(query5)
resultado5 = cur.fetchall()
print(f"Hay {resultado5[0][0]} eventos en curso en las fechas reservadas")

Hay 229 eventos en curso en las fechas reservadas


6. Identifica el día que más reservas se han hecho para nuestro hoteles

In [12]:
query6 = """ SELECT r.fecha_reserva, count(r.id_reserva)
                FROM reservas AS r
                JOIN hoteles AS h ON r.id_hotel = h.id_hotel
                GROUP BY r.fecha_reserva, h.competencia
                HAVING h.competencia = False
                ORDER BY count(r.id_reserva) DESC
                LIMIT 1
              ;"""
cur.execute(query6)
resultado6 = cur.fetchall()
print(f"El día que más reservas se han hecho para nuestros hoteles es el {resultado6[0][0]}")

El día que más reservas se han hecho para nuestros hoteles es el 2025-02-06


In [13]:
# Cerramos el cursor y la conexión
cur.close()
conn.close()